## 图像特征检测、描述与匹配

参考：

[OpenCV之特征检测](https://zhuanlan.zhihu.com/p/411370961)

[特征检测和特征匹配方法汇总](https://www.cnblogs.com/skyfsm/p/7401523.html)

[OpenCV-Python-Toturial page:178-218]()

## 定义

特征，也称 兴趣点 或 关键点，是图像中具有丰富信息的点，通常是图像中局部区域的显著变化点，如角点、边缘点等

<img src="./images/图像特征.png" alt="" style="height: 200px; width: auto;"/>

- 如图蓝色框中的区域是一个平面很难被找到和跟踪。无论你向那个方向移动蓝色框，长的都一样；
- 对于黑色框中的区域，它是一个边缘。如果你沿垂直方向移动会改变，但是如果沿水平方向移动就不会改变；
- 而红色框中的角点，无论你向那个方向移动，得到的结果都不同，这说明它是唯一的；
- 所以，基本上来说角点是一个好的图像特征。（不仅仅是角点，有些情况斑点也是好的图像特征）

我们称找到图像特征的技术为 特征检测 (Feature Detection)，描述图像特征的技术为 特征描述 (Feature Description)，将图像特征进行匹配的技术为 特征匹配 (Feature Matching)

## 角点检测

参考：

[OpenCV图像处理- 角点检测](https://zhuanlan.zhihu.com/p/68571164)

角点检测有两种经典方法：

- Harris角点检测
- Shi-Tomasi角点检测

### Harris角点检测

Chris_Harris 和 Mike_Stephens 在 1988 年的文章《A Combined Corner and Edge Detector》中提出了 Harris角点检测方法

其打分函数 R 定义为：

$$R = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2$$

在opencv中使用 cv2.cornerHarris() 函数实现 Harris角点检测:

函数原型：

```python
cv2.cornerHarris(src, blockSize, ksize, k) -> dst
```

参数说明：

- src：输入图像，要求为灰度图像，且数据类型为 float32
- blockSize：角点检测时考虑的邻域大小
- ksize：Sobel算子的孔径大小
- k：Harris检测方程中的自由参数，取值范围为 0.04~0.06，通常取 0.04

返回值：

- dst：检测结果图像，图像中每个像素的值表示该点为角点的可能性，值越大表示可能性越大

In [ ]:
import cv2
from my_function import imshow

img=cv2.imread("./images/blox.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

dst=cv2.cornerHarris(gray,2,3,0.04)
dst=cv2.dilate(dst,None) # 膨胀操作，标记更明显
img[dst>0.01*dst.max()]=[0,0,255] # 将角点标记为红色
imshow(Harris=img)

### Shi-Tomasi角点检测

Shi-Tomasi角点检测算法源自于论文《Good Features to Track》，基于Harris角点检测算法进行了改进

Shi-Tomasi的打分函数为：

$R = min(\lambda_1,\lambda_2)$，只有当$\lambda_1$和$\lambda_2$都大于某个阈值时，才认为该点是角点


在opencv中，可以使用函数`cv2.goodFeaturesToTrack()`来实现`Shi-Tomasi`角点检测

函数原型：

```python
cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]) → corners
```

参数说明：

- image：输入图像，必须为灰度图像
- maxCorners：要检测的最大角点数目
- qualityLevel：角点质量水平，取值范围为0到1，表示角点的最低质量
- minDistance：角点之间的最小距离
- corners：输出的角点坐标
- mask：可选的掩码图像，用于指定感兴趣的区域
- blockSize：计算角点时使用的邻域大小
- useHarrisDetector：是否使用Harris角点检测器，默认为False
- k：Harris角点检测器的自由参数

返回值：

- corners：检测到的角点坐标，格式为(N, 1, 2)的数组，其中N为检测到的角点数目

In [ ]:
img=cv2.imread("./images/blox.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

corners=cv2.goodFeaturesToTrack(gray,500,0.1,5)
# corners:(33, 1, 2)，第一维为角点集合，每个点坐标用1X2数组表示
for corner in corners:
    x,y=corner.ravel() # ravel()函数将多维数组转换为一维数组
    cv2.circle(img,(int(x),int(y)),3,(255,0,255),-1) # circle函数的圆心坐标需为整数

imshow(corners=img)

### 亚像素级角点检测

有时我们需要最大精度的角点检测。OpenCV为我们提供了函数`cv2.cornerSubPix()`，它可以提供亚像素级别的角点检测

函数原型：

```python
cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria) -> corners
```

参数说明：

- image：输入图像，必须为灰度图像
- corners：初始角点坐标
- winSize：搜索窗口的大小
- zeroZone：零区域的大小，一般设置为(-1,-1)
- criteria：迭代终止条件
  - cv2.TERM_CRITERIA_EPS：当角点位置的变化小于epsilon时停止迭代
  - cv2.TERM_CRITERIA_MAX_ITER：达到最大迭代次数时停止迭代
  - 一般将两者结合使用：`criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, max_iter, epsilon)`

返回值：

- corners：优化后的角点坐标

In [ ]:
img=cv2.imread("./images/blox.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

corners=cv2.goodFeaturesToTrack(gray,500,0.1,10)
corners=cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),
                         criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,40,0.001))
for corner in corners:
    x,y=corner.ravel()
    cv2.circle(img,(int(x),int(y)),3,(0,255,0),-1)
imshow(subpix_corners=img)

## 尺度不变特征变换（SIFT）